In [4]:
import pandas as pd
import sqlite3

# Paths to your CSV and Excel files
csv_file_path = '2020.csv'  
excel_file_path = 'Data_Mapping.xlsx'

# Load CSV data into a Pandas DataFrame
data_2020 = pd.read_csv(csv_file_path)

# Load Excel file containing the mapping (Entity -> Tables, Attribute -> Columns)
mapping_df = pd.read_excel(excel_file_path)

# Open SQLite connection
conn = sqlite3.connect('behavioral_risk_2020.db')

# Iterate over unique entities (table names) from the Excel file
for entity in mapping_df['Entity'].unique():
    # Get the columns (attributes) related to this entity (table)
    entity_columns = mapping_df[mapping_df['Entity'] == entity]['Attribute'].tolist()
    
    # Filter the entity columns that are available in the CSV data
    available_columns = [col for col in entity_columns if col in data_2020.columns]
    
    if available_columns:  # Proceed only if there are available columns
        # Subset the CSV data to include only these columns
        entity_data = data_2020[available_columns]
        
        # Store the subset data into a table in SQLite with the name of the entity
        entity_data.to_sql(entity, conn, if_exists='replace', index=False)
        print(f"Table '{entity}' created successfully with columns: {available_columns}")
    else:
        print(f"No matching columns found in the CSV for table '{entity}'.")

# Close the SQLite connection
conn.close()

print("All data successfully loaded into corresponding tables in SQLite.")


Table 'Alcohol' created successfully with columns: ['AVEDRNK3', 'DRNK3GE5', 'MAXDRNKS']
Table 'Cancer_test' created successfully with columns: ['HADHYST2', 'CNCRDIFF', 'CNCRAGE', 'CSRVTRT3', 'CSRVDOC1', 'CSRVSUM', 'CSRVRTRN', 'CSRVINST', 'CSRVINSR', 'CSRVDEIN', 'CSRVCLIN', 'CSRVPAIN', 'CSRVCTL2', 'PSATEST1']
Table 'Caregiver' created successfully with columns: ['CAREGIV1', 'CRGVREL4', 'CRGVLNG1', 'CRGVHRS1', 'CRGVPRB3', 'CRGVALZD', 'CRGVPER1', 'CRGVHOU1', 'CRGVEXPT']
Table 'childhood_exp' created successfully with columns: ['ACEDEPRS', 'ACEDRINK', 'ACEDRUGS', 'ACEPRISN', 'ACEDIVRC', 'ACEPUNCH', 'ACEHURT1', 'ACESWEAR', 'ACETOUCH', 'ACETTHEM', 'ACEHVSEX']
Table 'Children_details' created successfully with columns: ['RCSRLTN2', 'CASTHDX2', 'CASTHNO2', 'BIRTHSEX']
Table 'Cognitive_Decline' created successfully with columns: ['CIMEMLOS', 'CDHOUSE', 'CDASSIST', 'CDHELP', 'CDSOCIAL', 'CDDISCUS']
Table 'Common_Variables ' created successfully with columns: ['_STATE', 'FMONTH', 'IDATE', 'IMONTH

In [7]:
import os

# Set the environment variable within the notebook
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "top-suprstate-438620-h9-7a2effbc371a.json"

# Now proceed with Google Cloud Storage client initialization
from google.cloud import storage

# Instantiate a Google Cloud Storage client
client = storage.Client()

# List the buckets in your project
buckets = list(client.list_buckets())
print(buckets)


[<Bucket: gcf-v2-sources-892122457228-us-central1>, <Bucket: gcf-v2-uploads-892122457228-us-central1>, <Bucket: on-prem-ingestion-data>]


In [10]:
import sqlite3
import pandas as pd
from google.cloud import storage
import io

# Google Cloud Storage configuration
bucket_name = 'on-prem-ingestion-data'  
client = storage.Client()
bucket = client.get_bucket(bucket_name)

# Connect to SQLite database
conn = sqlite3.connect('behavioral_risk_2020.db')

# Get a list of all tables in the SQLite database
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)

# Iterate through each table and upload data directly to GCS
for table_name in tables['name']:
    # Read the table into a Pandas DataFrame
    table_data = pd.read_sql(f'SELECT * FROM {table_name}', conn)
    
    # Create an in-memory buffer for CSV data
    csv_buffer = io.StringIO()
    
    # Write the DataFrame to the buffer in CSV format
    table_data.to_csv(csv_buffer, index=False)
    
    # Set the destination path in GCS (adjust the path as needed)
    destination_blob_name = f'sql_tables/{table_name}.csv'
    blob = bucket.blob(destination_blob_name)
    
    # Upload the buffer's content to GCS
    blob.upload_from_string(csv_buffer.getvalue(), content_type='text/csv')
    
    print(f"Table '{table_name}' uploaded to GCS as '{destination_blob_name}'")

# Close SQLite connection
conn.close()

print("All tables uploaded successfully to GCS.")


Table 'Alcohol' uploaded to GCS as 'sql_tables/Alcohol.csv'
Table 'Cancer_test' uploaded to GCS as 'sql_tables/Cancer_test.csv'
Table 'Caregiver' uploaded to GCS as 'sql_tables/Caregiver.csv'
Table 'childhood_exp' uploaded to GCS as 'sql_tables/childhood_exp.csv'
Table 'Children_details' uploaded to GCS as 'sql_tables/Children_details.csv'
Table 'Cognitive_Decline' uploaded to GCS as 'sql_tables/Cognitive_Decline.csv'


DatabaseError: Execution failed on sql 'SELECT * FROM Common_Variables ': no such table: Common_Variables